## Chapter6
D3.js in Actionの6章の勉強ノートです。

In [11]:
%load_ext sage
from IPython.core.display import HTML
from string import Template
import json
import nvd3
nvd3.ipynb.initialize_javascript(use_remote=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
%%writefile css/network.css
.grid {
              stroke: black;
              stroke-width: 1px;
fill: red; }
        .arc {
               stroke: black;
fill: none;
}
.node {
              fill: lightgray;
              stroke: black;
              stroke-width: 1px;
        }
        circle.active {
fill: red;
        }
        path.active {
stroke: red;
}

Overwriting css/network.css


In [13]:
%%writefile data/edgelist.csv
source,target,weight
sam,pris,1
roy,pris,5
roy,sam,1
tully,pris,5
tully,kim,3
tully,pat,1
tully,mo,3
kim,pat,2
kim,mo,1
mo,tully,7
mo,pat,1
mo,pris,1
pat,tully,1
pat,kim,2
pat,mo,5
lee,al,3

Overwriting data/edgelist.csv


In [14]:
%%writefile data/nodelist.csv
id,followers,following
sam,17,500
roy,83,80
pris,904,15
tully,7,5
kim,11,50
mo,80,85
pat,150,300
lee,38,7
al,12,12

Overwriting data/nodelist.csv


In [36]:
%%HTML
<!-- CSSの読み込み　-->
<link type="text/css" rel="stylesheet" href="css/network.css" />
<script src="js/d3.min.js" type="text/javascript"></script>

<div id="viz">
  <svg style="width: 350px; height:350px; border:1px lightgray solid;" >
  </svg>
</div>

In [39]:
%%javascript
var nodes;
var edges;
var target="#vis";

var marker = d3.select("svg").append('defs')
    .append('marker')
    .attr("id", "Triangle")
    .attr("refX", 12)
    .attr("refY", 6)
    .attr("markerUnits", 'userSpaceOnUse')
    .attr("markerWidth", 12)
    .attr("markerHeight", 18)
    .attr("orient", 'auto')
    .append('path')
    .attr("d", 'M 0 0 12 6 0 12 3 6');

d3.csv("data/nodelist.csv",function(error,data) { 
    nodes = data;
    d3.csv("data/edgelist.csv", function(error,data) {
        edges = data;
        createForceLayout(nodes,edges);
    });
});

function createForceLayout(nodes,edges) {
  var nodeHash = {};      
  for (var x in nodes) {
    nodeHash[nodes[x].id] = nodes[x];
  }
  for (var x in edges) {
    edges[x].weight = parseInt(edges[x].weight);
    edges[x].source = nodeHash[edges[x].source];
    edges[x].target = nodeHash[edges[x].target];
  }
  var weightScale = d3.scale.linear().domain(d3.extent(edges, function(d) {return d.weight})).range([.1,1])
  var force = d3.layout.force()
  .charge(function (d) {return d.weight * -500})
  .gravity(.3)
  .size([500,500]).nodes(nodes)
  .links(edges).on("tick", forceTick);
    
  d3.select("svg").selectAll("line.link").data(edges, function (d) {return d.source.id + "-" + d.target.id}).enter()
  .append("line")
  .attr("class", "link")
  .style("stroke", "black")
  .style("opacity", .5)
  .style("stroke-width", function(d) {return d.weight});

  var nodeEnter = d3.select("svg").selectAll("g.node").data(nodes, function (d) {return d.id}).enter()
  .append("g")
  .attr("class", "node")
  .call(force.drag())
  .on("click", fixNode);

  function fixNode(d) {
    d3.select(this).select("circle").style("stroke-width", 4);
    d.fixed = true;
  }

  nodeEnter.append("circle")
  .attr("r", 5)
  .style("fill", "lightgray")
  .style("stroke", "black")
  .style("stroke-width", "1px");

  nodeEnter.append("text")
  .style("text-anchor", "middle")
  .attr("y", 15)
  .text(function(d) {return d.id})
d3.selectAll("line").attr("marker-end", "url(#Triangle)");
  force.start();

  function forceTick() {
  d3.selectAll("line.link")
  .attr("x1", function (d) {return d.source.x})
  .attr("x2", function (d) {return d.target.x})
  .attr("y1", function (d) {return d.source.y})
  .attr("y2", function (d) {return d.target.y});

  d3.selectAll("g.node")
  .attr("transform", function (d) {return "translate("+d.x+","+d.y+")"})
  }
}

<IPython.core.display.Javascript object>